In [2]:
import os
import numpy as np
import torch
from PIL import Image
from torchvision.models import detection
import torchvision
from torchvision import datasets, models, transforms
import cv2
import matplotlib.pyplot as plt
import torchvision
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import time
from tempfile import TemporaryDirectory

In [4]:
from metrics import *

# Initialize the Weight Transforms
weights = models.ResNet50_Weights.DEFAULT
preprocess = weights.transforms()
model = models.resnet50(weights=weights)

# Set model to eval mode
model.eval()

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\David/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [01:40<00:00, 1.02MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
from dataloader import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Some code is from here: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
# Data augmentation and normalization for training
# Just normalization for validation
# data_transforms = {
#     'train': transforms.Compose([
#         transforms.RandomResizedCrop(224),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     'val': transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
# }
# maybe we need to change the transformations. Maybe the model weights were calculated using data with different transformations. Or maybe we need to finetune each model to our transformations

dataset_paths = {
    "imagenet": "E:/MLreliability_project/data/ILSVRC2012_images_val",
    "imagenet-v2-threshold": "E:/MLreliability_project/data/imagenetv2-threshold0.7",
    "imagenet-r": "E:/MLreliability_project/data/imagenet-r",
    "imagenet-sketch": "E:/MLreliability_project/data/sketch",
    "imagenet-vid": "E:/MLreliability_project/data/imagenet_vid_ytbb_robust",
    "imagenet-v2-matched": "E:/MLreliability_project/data/imagenetv2-matched-frequency",
    "imagenet-v2-top": "E:/MLreliability_project/data/imagenetv2-top-images",
    "stl": "E:/MLreliability_project/data/stl10_binary"
}

if 'David' in os.getcwd():  # checking in whose computer we are
    val_dir = dataset_paths["imagenet-r"]
else:
    print("The path of the files is in Amitay's computer")

batch_size = 4

# imagenet_data = torchvision.datasets.ImageNet('path/to/imagenet_root/')

# they used ImageNet as the base dataset
# they used ImageNet-V2, ImageNet-VidRobust, ImageNet-Rendition and ImageNet-Sketch for ImageNet-Adversarial for natural OOD shift. They also used ImageNet-Adversarial as a natural OOD shift. This dataset was examined separately
# they used ImageNet-C as a synthetic OOD shift
# They also somehow used ImageNet-Validation



# trainset = torchvision.datasets.ImageNet(root=train_dir, train=True,
#                                         download=True, transform=data_transforms)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
#                                           shuffle=True, num_workers=2)
try:
    create_dataloader(dataset_path=val_dir, batch_size=batch_size, preprocess_func=preprocess)
except:
    valset = torchvision.datasets.ImageNet(root=val_dir, train=False,
                                           download=False, transform=preprocess)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                             shuffle=False, num_workers=2)

# not really our classes so should be changed or deleted
# classes = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# image_datasets = {'train': trainset,
#                   'test': valset}
# dataloaders = {'train': trainloader,
#                   'test': valloader}

# dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
# class_names = image_datasets['train'].classes

# the models that were used in the paper: resnet18, resnet34, resnet50, resnet101, resnet152, vgg19, alexnet, resnext101_32x8d or resnext101_64x4d (they didn't specify which resnet101 they used. Also used wide_resnet101_2 and AugMix, DeepAugment, AM-DeepAugment and Deep Ensembles
# resnet18 = models.resnet18(weights='IMAGENET1K_V1')

RuntimeError: The archive ILSVRC2012_devkit_t12.tar.gz is not present in the root directory or is corrupted. You need to download it externally and place it in E:/MLreliability_project/data/ILSVRC2010_images_test.

In [ ]:
import subprocess
subprocess.call(['sh', './test.sh']) # Thanks @Jim Dennis for suggesting the []

In [ ]:
calculate_confidences(model, valloader)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    # The function does training including validation
    # it's taken from here:
    # https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html#training-the-model

    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model


def AC(model, dataset):
    # the function does inference and then calculates the average confidence of the model on the dataset

    was_training = model.training
    model.eval()

    loader = torch.utils.data.DataLoader(trainset, batch_size=len(dataset),
                                          shuffle=True, num_workers=2)  # a dataloader that loads all the samples at once
    inputs, _ = next(iter(loader))  # all samples without labels

    with torch.no_grad():
        outputs = model(inputs)
        probmax = torch.max(outputs, axis = 0)  # the score(s) of the class with the highest score(s)
        average_confidence = torch.mean(probmax)  # calculating average confidence

        model.train(mode=was_training)

    return average_confidence


def DOC(model, in_distribution_dataset, out_distribution_dataset):
    # the function calculates the average confidence of the model on two datasets and then calculates the DoC (difference of confidences) over the two datasets

    DoC = AC(model, in_distribution_dataset) - AC(model, out_distribution_dataset)
    return DoC

In [ ]:
model_ft = resnet18
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

In [5]:
# I found an implementation of DoC but they did a bit different from what I did. They subtracted the accuracy from the average confidence and said that the result is the DoC. I don't know why: https://github.com/ZerojumpLine/ModelEvaluationUnderClassImbalance/blob/main/Prostate.ipynb

In [ ]:
from __future__ import print_function

import sys
import os, sys, tarfile, errno
import numpy as np
import matplotlib.pyplot as plt

if sys.version_info >= (3, 0, 0):
    import urllib.request as urllib # ugly but works
else:
    import urllib

try:
    from imageio import imsave
except:
    from scipy.misc import imsave

print(sys.version_info)

# image shape
HEIGHT = 96
WIDTH = 96
DEPTH = 3

# size of a single image in bytes
SIZE = HEIGHT * WIDTH * DEPTH

# path to the directory with the data
DATA_DIR = './data'

# url of the binary data
DATA_URL = 'http://ai.stanford.edu/~acoates/stl10/stl10_binary.tar.gz'

# path to the binary train file with image data
DATA_PATH = './data/stl10_binary/train_X.bin'

# path to the binary train file with labels
LABEL_PATH = './data/stl10_binary/train_y.bin'

def read_labels(path_to_labels):
    """
    :param path_to_labels: path to the binary file containing labels from the STL-10 dataset
    :return: an array containing the labels
    """
    with open(path_to_labels, 'rb') as f:
        labels = np.fromfile(f, dtype=np.uint8)
        return labels


def read_all_images(path_to_data):
    """
    :param path_to_data: the file containing the binary images from the STL-10 dataset
    :return: an array containing all the images
    """

    with open(path_to_data, 'rb') as f:
        # read whole file in uint8 chunks
        everything = np.fromfile(f, dtype=np.uint8)

        # We force the data into 3x96x96 chunks, since the
        # images are stored in "column-major order", meaning
        # that "the first 96*96 values are the red channel,
        # the next 96*96 are green, and the last are blue."
        # The -1 is since the size of the pictures depends
        # on the input file, and this way numpy determines
        # the size on its own.

        images = np.reshape(everything, (-1, 3, 96, 96))

        # Now transpose the images into a standard image format
        # readable by, for example, matplotlib.imshow
        # You might want to comment this line or reverse the shuffle
        # if you will use a learning algorithm like CNN, since they like
        # their channels separated.
        images = np.transpose(images, (0, 3, 2, 1))
        return images


def read_single_image(image_file):
    """
    CAREFUL! - this method uses a file as input instead of the path - so the
    position of the reader will be remembered outside of context of this method.
    :param image_file: the open file containing the images
    :return: a single image
    """
    # read a single image, count determines the number of uint8's to read
    image = np.fromfile(image_file, dtype=np.uint8, count=SIZE)
    # force into image matrix
    image = np.reshape(image, (3, 96, 96))
    # transpose to standard format
    # You might want to comment this line or reverse the shuffle
    # if you will use a learning algorithm like CNN, since they like
    # their channels separated.
    image = np.transpose(image, (2, 1, 0))
    return image


def plot_image(image):
    """
    :param image: the image to be plotted in a 3-D matrix format
    :return: None
    """
    plt.imshow(image)
    plt.show()

def save_image(image, name):
    imsave("%s.png" % name, image, format="png")

def download_and_extract():
    """
    Download and extract the STL-10 dataset
    :return: None
    """
    dest_directory = DATA_DIR
    if not os.path.exists(dest_directory):
        os.makedirs(dest_directory)
    filename = DATA_URL.split('/')[-1]
    filepath = os.path.join(dest_directory, filename)
    if not os.path.exists(filepath):
        def _progress(count, block_size, total_size):
            sys.stdout.write('\rDownloading %s %.2f%%' % (filename,
                float(count * block_size) / float(total_size) * 100.0))
            sys.stdout.flush()
        filepath, _ = urllib.urlretrieve(DATA_URL, filepath, reporthook=_progress)
        print('Downloaded', filename)
        tarfile.open(filepath, 'r:gz').extractall(dest_directory)

def save_images(images, labels):
    print("Saving images to disk")
    i = 0
    for image in images:
        label = labels[i]
        directory = './img/' + str(label) + '/'
        try:
            os.makedirs(directory, exist_ok=True)
        except OSError as exc:
            if exc.errno == errno.EEXIST:
                pass
        filename = directory + str(i)
        print(filename)
        save_image(image, filename)
        i = i+1


# download data if needed
download_and_extract()

# test to check if the image is read correctly
with open(DATA_PATH) as f:
    image = read_single_image(f)
    plot_image(image)

# test to check if the whole dataset is read correctly
images = read_all_images(DATA_PATH)
print(images.shape)

labels = read_labels(LABEL_PATH)
print(labels.shape)

# save images to disk
save_images(images, labels)
